In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

from nobita.models import get_sequence_model


try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
# APEX_AVAILABLE = False

In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

Reference:

    * https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict):
        super().__init__()
        self.entries_dict = entries_dict
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx].sum(), #input_lengths
            {
                "label": self.entries_dict["label"][idx], 
                "logits": self.entries_dict["logits"][idx]
            }
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict), batch_size=64, drop_last=False)

In [7]:
ALPHA = 0.
DISTILL_OBJECTIVE = torch.nn.MSELoss()

def distill_loss(logits, targets):
    distill_part = DISTILL_OBJECTIVE(
        logits.reshape(-1), targets["logits"].reshape(-1)
    ) / 2
    classification_part = F.cross_entropy(
        logits, targets["label"]
    )
    return ALPHA * classification_part + (1-ALPHA) * distill_part

In [8]:
bert_model = BertForSequenceClassification.from_pretrained(str(CACHE_DIR / "sst2_bert_uncased")).cpu()

In [9]:
bert_model.bert.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [10]:
# Note: apex does not support weight dropping
model = get_sequence_model(
    voc_size=bert_model.bert.embeddings.word_embeddings.weight.shape[0],
    emb_size=bert_model.bert.embeddings.word_embeddings.weight.shape[1],
    pad_idx = 0,
    dropoute = 0,
    rnn_hid = 768,
    rnn_layers = 2,
    bidir = True,
    dropouth = 0.25,
    dropouti = 0.25,
    wdrop = 0,
    unit_type = "gru",
    fcn_layers = [512, 2],
    fcn_dropouts = [0.25, 0.25],
    use_attention = True
)

In [11]:
model

SequenceModel(
  (embeddings): BasicEmbeddings(
    (encoder): Embedding(30522, 768, padding_idx=0)
  )
  (encoder): RNNStack(
    (rnns): ModuleList(
      (0): GRU(768, 384, bidirectional=True)
      (1): GRU(768, 384, bidirectional=True)
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
    )
  )
  (fcn): AttentionFCN(
    (attention): Attention(768, return attention=False)
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=768, out_features=512, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): LinearBlock(
        (lin): Linear(in_features=512, out_features=2, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
)

In [12]:
# Copy the embedding weights to the LSTM model
try:
    model.embeddings.encoder.emb.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data
except:
    model.embeddings.encoder.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data

In [13]:
# Freeze the embedding layer
for param in model.embeddings.encoder.parameters():
    param.requires_grad = False

In [14]:
model = model.cuda()

In [15]:
# Use only leaf tensors
parameters = [x for x in model.parameters() if x.is_leaf and x.requires_grad]

In [16]:
del bert_model

In [17]:
optimizer = torch.optim.Adam(parameters, lr=1e-3, betas=(0.8, 0.99))
# optimizer = torch.optim.RMSprop(parameters, lr=0.01)

In [18]:
if APEX_AVAILABLE:
    model, optimizer = amp.initialize(
        model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
class TransposeCallback(Callback):
    def on_batch_inputs(self, bot, input_tensors, targets):
        input_tensors = [input_tensors[0].transpose(1, 0), input_tensors[1]]
        return input_tensors, targets

In [20]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [21]:
total_steps = len(train_loader) * 10

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints,
    TransposeCallback()
]
    
bot = BaseBot(
    log_dir = CACHE_DIR / "distill_logs",
    model=model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=distill_loss,
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(DistillTop1Accuracy(),)
)

[INFO][06/15/2020 20:56:59] SEED: 9293
[INFO][06/15/2020 20:56:59] # of parameters: 29,156,610
[INFO][06/15/2020 20:56:59] # of trainable parameters: 5,715,714


In [22]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][06/15/2020 20:56:59] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.8, 0.99)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
[INFO][06/15/2020 20:56:59] Batches per epoch: 1053
[INFO][06/15/2020 20:56:59] ====================Epoch 1====================


10530
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/data/miniconda3/envs/rapids/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


[INFO][06/15/2020 20:57:01] Step   105 | loss 5.88067631 | lr: 5.94e-05 | 0.020s per step
[INFO][06/15/2020 20:57:03] Step   210 | loss 4.98701024 | lr: 1.09e-04 | 0.018s per step
[INFO][06/15/2020 20:57:05] Step   315 | loss 4.03495032 | lr: 1.58e-04 | 0.020s per step
[INFO][06/15/2020 20:57:07] Step   420 | loss 3.62690906 | lr: 2.08e-04 | 0.022s per step
[INFO][06/15/2020 20:57:09] Step   525 | loss 3.37542540 | lr: 2.57e-04 | 0.020s per step
[INFO][06/15/2020 20:57:10] Metrics at step 526:
[INFO][06/15/2020 20:57:10] loss: 2.32088318
[INFO][06/15/2020 20:57:10] accuracy: 73.62%
[INFO][06/15/2020 20:57:12] Step   630 | loss 3.17826421 | lr: 3.06e-04 | 0.021s per step
[INFO][06/15/2020 20:57:14] Step   735 | loss 2.99851566 | lr: 3.56e-04 | 0.020s per step
[INFO][06/15/2020 20:57:16] Step   840 | loss 2.81948140 | lr: 4.05e-04 | 0.019s per step
[INFO][06/15/2020 20:57:18] Step   945 | loss 2.58947242 | lr: 4.54e-04 | 0.020s per step
[INFO][06/15/2020 20:57:20] Step  1050 | loss 2.444

...

[INFO][06/15/2020 20:58:26] Step  4515 | loss 0.90879787 | lr: 8.12e-04 | 0.019s per step
[INFO][06/15/2020 20:58:28] Step  4620 | loss 0.91673689 | lr: 7.97e-04 | 0.019s per step
[INFO][06/15/2020 20:58:30] Step  4725 | loss 0.92463056 | lr: 7.81e-04 | 0.020s per step
[INFO][06/15/2020 20:58:31] Metrics at step 4734:
[INFO][06/15/2020 20:58:31] loss: 1.03975216
[INFO][06/15/2020 20:58:31] accuracy: 86.47%
[INFO][06/15/2020 20:58:33] Step  4830 | loss 0.90784699 | lr: 7.64e-04 | 0.021s per step
[INFO][06/15/2020 20:58:35] Step  4935 | loss 0.83595148 | lr: 7.47e-04 | 0.019s per step
[INFO][06/15/2020 20:58:37] Step  5040 | loss 0.84228908 | lr: 7.30e-04 | 0.020s per step
[INFO][06/15/2020 20:58:39] Step  5145 | loss 0.84274006 | lr: 7.13e-04 | 0.020s per step
[INFO][06/15/2020 20:58:41] Step  5250 | loss 0.82992634 | lr: 6.95e-04 | 0.020s per step
[INFO][06/15/2020 20:58:41] Metrics at step 5260:
[INFO][06/15/2020 20:58:41] loss: 0.93030948
[INFO][06/15/2020 20:58:41] accuracy: 85.09%


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


[INFO][06/15/2020 20:59:45] Step  8505 | loss 0.48967614 | lr: 1.36e-04 | 0.022s per step
[INFO][06/15/2020 20:59:47] Step  8610 | loss 0.50541305 | lr: 1.23e-04 | 0.020s per step
[INFO][06/15/2020 20:59:49] Step  8715 | loss 0.49594498 | lr: 1.10e-04 | 0.019s per step
[INFO][06/15/2020 20:59:51] Step  8820 | loss 0.50690323 | lr: 9.85e-05 | 0.019s per step
[INFO][06/15/2020 20:59:53] Step  8925 | loss 0.46257111 | lr: 8.71e-05 | 0.019s per step
[INFO][06/15/2020 20:59:54] Metrics at step 8942:
[INFO][06/15/2020 20:59:54] loss: 0.84927264
[INFO][06/15/2020 20:59:54] accuracy: 88.30%
[INFO][06/15/2020 20:59:56] Step  9030 | loss 0.48975894 | lr: 7.64e-05 | 0.021s per step
[INFO][06/15/2020 20:59:58] Step  9135 | loss 0.48621000 | lr: 6.63e-05 | 0.019s per step
[INFO][06/15/2020 21:00:00] Step  9240 | loss 0.48479956 | lr: 5.69e-05 | 0.019s per step
[INFO][06/15/2020 21:00:02] Step  9345 | loss 0.46938192 | lr: 4.82e-05 | 0.019s per step
[INFO][06/15/2020 21:00:04] Step  9450 | loss 0.47

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


[INFO][06/15/2020 21:00:24] Training finished. Best step(s):
[INFO][06/15/2020 21:00:24] loss: 0.84927264 @ step 8942
[INFO][06/15/2020 21:00:24] accuracy: 88.76% @ step 9994


In [23]:
# TARGET_DIR = CACHE_DIR / "sst2_bert_uncased"
# TARGET_DIR.mkdir(exist_ok=True)
# bot.model.save_pretrained(TARGET_DIR)

In [24]:
bot.eval(valid_loader)

{'loss': (0.8493302934760347, '0.84933029'),
 'accuracy': (-0.8830275229357798, '88.30%')}

In [25]:
bot.eval(test_loader)

{'loss': (0.9584249192421589, '0.95842492'),
 'accuracy': (-0.8715596330275229, '87.16%')}

In [26]:
# tokenizer.pad_token_id